<a href="https://colab.research.google.com/github/seanmcalevey/quick-portfolio/blob/master/basic_movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Basic Imports (Colab)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Main DF

In [2]:
drive_path = '/content/drive/My Drive/'
folder_path = 'kaggle_datasets/movies_dataset/'
file_path = 'movies_metadata.csv'
metadata_df = pd.read_csv(drive_path + folder_path + file_path)
metadata_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Trim Down Dataset Features

In [3]:
cols_to_keep = ['original_title', 'overview', 'original_language', 'vote_count']
overview_df = metadata_df[cols_to_keep]
overview_df = overview_df[overview_df['original_language'] == 'en'] # only english movies
overview_df = overview_df.drop('original_language', axis=1) # drop language feature
overview_df = overview_df.dropna(subset=['overview']) # drop nans
overview_df.shape

(32200, 3)

### Drop Movies with Few Views

In [27]:
overview_df = overview_df.dropna()
t = overview_df['vote_count']
q = 0.75
min_votes = np.quantile(t, q)
print(f'Film needs to be voted on {min_votes} many times.')
overview_df = overview_df[overview_df['vote_count'] > min_votes]
overview_df = overview_df.reset_index(drop=True)

### Text Cleaning

In [28]:
import re

cleaned_text = [re.sub('[^A-Za-z\s]', '', text) for text in overview_df['overview']]
cleaned_text[:5] # show first five processed texts

['Led by Woody Andys toys live happily in his room until Andys birthday brings Buzz Lightyear onto the scene Afraid of losing his place in Andys heart Woody plots against Buzz But when circumstances separate Buzz and Woody from their owner the duo eventually learns to put aside their differences',
 'When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world they unwittingly invite Alan  an adult whos been trapped inside the game for  years  into their living room Alans only hope for freedom is to finish the game which proves risky as all three find themselves running from giant rhinoceroses evil monkeys and other terrifying creatures',
 'A family wedding reignites the ancient feud between nextdoor neighbors and fishing buddies John and Max Meanwhile a sultry Italian divorce opens a restaurant at the local bait shop alarming the locals who worry shell scare the fish away But shes less interested in seafood than she is in cooking up a hot time wi

### Tfidf Vectorization

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

max_features = 5000

vec = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=max_features)
tfidf_matrix = vec.fit_transform(cleaned_text)
tfidf_matrix

<7999x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 158306 stored elements in Compressed Sparse Row format>

### Pairwise Cosine Similarity Score Matrix

In [30]:
from sklearn.metrics.pairwise import linear_kernel

cos_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

### Title-Index Lookup Table

In [31]:
title_lookup = pd.Series(overview_df.index, index=overview_df['original_title'])
title_lookup.head()

original_title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Father of the Bride Part II    3
Heat                           4
dtype: int64

### Recommender Function

In [33]:
def get_recommendations(title, df=overview_df, cos_matrix=cos_matrix):
  idx = title_lookup.loc[f'{title}']
  tuple_list = list(enumerate(cos_matrix[idx]))
  sorted_list = sorted(tuple_list, key=lambda x: x[1], reverse=True) # sort list
  top_10_scores = sorted_list[1:11] # get top 10
  idxs = [tupl[0] for tupl in top_10_scores] # splice tupls to get indices
  similarity_scores = [tupl[1] for tupl in top_10_scores] # splice tupls to get similarity scores
  return_df = df.iloc[idxs,:]
  return_df['similarity_score'] = similarity_scores
  return_df = return_df[['original_title', 'overview', 'similarity_score']]
  return return_df

### Sample Titles

In [37]:
list_of_movies = ['The Departed', 'Phantom Thread', 'Gladiator', 'Mean Girls']

for movie in list_of_movies:
  print(movie, '\n')
  print(get_recommendations(movie_title))
  print('\n')

The Departed 

                original_title  ... similarity_score
717              Donnie Brasco  ...         0.251885
1399          Mickey Blue Eyes  ...         0.177849
6677                     Blood  ...         0.174499
1102                     Ronin  ...         0.168606
3606                   Hostage  ...         0.167071
2709  Ordinary Decent Criminal  ...         0.155123
5286         Kill the Irishman  ...         0.151247
3422           Charley Varrick  ...         0.150342
7107                Black Mass  ...         0.145689
2004      Beverly Hills Cop II  ...         0.143087

[10 rows x 3 columns]


Phantom Thread 

                original_title  ... similarity_score
717              Donnie Brasco  ...         0.251885
1399          Mickey Blue Eyes  ...         0.177849
6677                     Blood  ...         0.174499
1102                     Ronin  ...         0.168606
3606                   Hostage  ...         0.167071
2709  Ordinary Decent Criminal  ...       

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do